# 스포츠 강좌 챗봇

In [ ]:
# import 하기
import requests
import gradio as gr
from flask import Flask, render_template_string
import threading

In [ ]:
## 이전 내용 기록
def get_history_message(histories):
    history_list = list()
    
    history_length = 5
    history_index = 0
    
    for history in histories:

        if history_index >= history_length:
            break
        
        message1 = history[0]
        message2 = history[1]
        
        history_list.append({
            "role": 'assistant',
            "content": message1
        })
        history_list.append({
            "role": "assistant",
            "content": message2
        })
        
        history_index += 1
    return history_list

In [ ]:
## request 및 API 설정 및 기본 설정
def request_gpt(prompt, history_list):
    
    # endpoint
    endpoint = ""    
    
    ai_search_endpoint = "" # azure search service URL
    ai_search_key = "" # Azure Search Key
    ai_search_index = "sportspot2" # Azure search index
    
    # method(post)
    headers = {
        "Content-Type" : "application/json",
        "api-key": ""
    }
    message_list = list()
    
    #system
    message_list.append({
        "role": "system",
        "content": "너는 이제부터 전국체육시설을 친절하게 말해주는 서비스직 에이스야. 앞으로 사용자가 질문을 하면 잘 대답해줘야해. 또한 사용자가 무슨 지역에 살고 무슨 스포츠를 하고싶어하는지 그리고 어떤 목적을 가지고 질문을 하면 잘 파악해서 자세하게 알려줘야해."
    })
    # 이전 메세지
    message_list.extend(history_list)
    # prompt
    message_list.append({
        "role": "user",
        "content": prompt
    })
    payload = {
        "messages": message_list,
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 800,
        "data_sources": [
        {
            "type": "azure_search",
            "parameters": {
                "endpoint": ai_search_endpoint,
                "semantic_configuration": "sport-segmantic",
                "query_type": "semantic",
                "strictness": 3,
                "top_n_documents": 5,
                "key": ai_search_key,
                "indexName": ai_search_index
            }
        }
        ]
    }
    
    response = requests.post(endpoint, headers=headers, json=payload)
    
    if response.status_code == 200:
        response_json = response.json()
        content = response_json["choices"][0]["message"]["content"]
        return content
    else:
        return f"{response.status_code}, {response.text}"

In [ ]:
app = Flask(__name__)

def click_send(prompt, histories):
    history_list = get_history_message(histories=histories)
    response_text = request_gpt(prompt, history_list)
    histories.append((prompt, response_text))
    return histories, ""


with gr.Blocks() as demo:
  
    # chatbot
    chatbot = gr.Chatbot(label="스포츠 채팅 내용")
      
    with gr.Row():
        gr.Markdown("## 🏅 스포츠 어드바이저 챗봇")
        gr.Markdown("## 스포츠 일정, 장소 추천 및 운동 팁을 제공합니다.")
    
    
    # input
    with gr.Row():
        input_textbox = gr.Textbox(lines=2, placeholder="궁금사항을 입력하세요 (예: 00지역 원하는 스포츠 체육시설 또는 강좌 프로그램).", show_label=False)
        send_button = gr.Button("전송")
       
   
    # submit
    input_textbox.submit(fn=click_send, inputs=[input_textbox, chatbot], outputs=[chatbot, input_textbox])
    send_button.click(fn=click_send, inputs=[input_textbox, chatbot], outputs=[chatbot, input_textbox])
    

# Flask 라우팅 설정
@app.route("/")
def index():
    return render_template_string(
        """
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>스포츠 어드바이저 챗봇</title>
            <link rel="stylesheet" type="text/css" href="/static/styles.css">
        </head>
        <body>
            <div id="gradio-interface"></div>
            {{ gradio_script|safe }}
        </body>
        </html>
        """,
        gradio_content=demo.render()
    )


@app.route("/gradio")
def gradio_interface():
    return demo.launch(server_name="0.0.0.0", server_port=7860, inline=False, share=False)

if __name__ == "__main__":
    app.run(debug=True)